In [3]:
import pandas as pd
import numpy as np

# Set the dataset and norm
dataset = "kitti-2015"
norm = "inf"

# Define the datasets and corresponding model names
kitti_model_names = [
    "raft", "gma", "rpknet", "ccmr", "craft", "csflow", "dicl", "dip", 
    "fastflownet", "maskflownet", "flow1d", "flowformer", "flowformer++", 
    "gmflow", "gmflownet", "hd3", "irr_pwc", "liteflownet", "liteflownet2", 
    "liteflownet3_pseudoreg", "llaflow", "matchflow", "ms_raft+", 
    "rapidflow", "scopeflow", "scv4", "separableflow", "skflow", 
    "starflow", "videoflow_bof"
]

sintel_model_names = [
    "raft", "pwcnet", "gma", "rpknet", "ccmr", "craft", "dicl", "dip", 
    "fastflownet", "maskflownet", "maskflownet_s", "flow1d", "flowformer", 
    "flowformer++", "gmflow", "hd3", "liteflownet", 
    "liteflownet2", "liteflownet3", "llaflow", "matchflow", 
    "ms_raft+", "rapidflow", "scopeflow", "scv4", "separableflow", 
    "skflow", "starflow", "videoflow_bof"
]



# Choose the model list based on the dataset
if dataset == "kitti-2015":
    model_names = kitti_model_names
elif dataset in ["sintel-clean", "sintel-final"]:
    model_names = sintel_model_names
else:
    raise ValueError(f"Dataset '{dataset}' is not recognized.")

# Read the CSV file
df = pd.read_csv('iteration_df.csv', delimiter=',')

# Sort by 'model' and filter by 'dataset'
df_sorted = df.sort_values(by='model')
df_sorted = df_sorted[df_sorted["dataset"] == dataset]

# Define conditions
conditions = (
    (df_sorted['epsilon'].isin([0, 0.0314, 0.0157])) & 
    (df_sorted['alpha'] == 0.01) &
    (df_sorted['iterations'] == 20) &
    (df_sorted['norm'] == norm) &
    (df_sorted['loss'] == 'epe') &
    (df_sorted['targeted'].isin([True, False])) &
    (df_sorted['target'].isin(["zero", "negative"])) | (df_sorted['target'].isna())
)

# Define expected combinations, including the new criteria
expected_combinations = [
    {'targeted': True, 'target': 'negative', 'attack': 'pgd', 'epsilon': 0.0314, 'alpha': 0.01, 'iterations': 20, 'norm': norm, 'loss': 'epe'},
    {'targeted': True, 'target': 'zero', 'attack': 'pgd', 'epsilon': 0.0314, 'alpha': 0.01, 'iterations': 20, 'norm': norm, 'loss': 'epe'},
    {'targeted': True, 'target': 'negative', 'attack': 'cospgd', 'epsilon': 0.0314, 'alpha': 0.01, 'iterations': 20, 'norm': norm, 'loss': 'epe'},
    {'targeted': True, 'target': 'zero', 'attack': 'cospgd', 'epsilon': 0.0314, 'alpha': 0.01, 'iterations': 20, 'norm': norm, 'loss': 'epe'},
    {'targeted': True, 'target': 'negative', 'attack': 'bim', 'epsilon': 0.0314, 'alpha': 0.01, 'iterations': 20, 'norm': norm, 'loss': 'epe'},
    {'targeted': True, 'target': 'zero', 'attack': 'bim', 'epsilon': 0.0314, 'alpha': 0.01, 'iterations': 20, 'norm': norm, 'loss': 'epe'},
    {'targeted': False, 'target': np.nan, 'attack': 'pgd', 'epsilon': 0.0157, 'alpha': 0.01, 'iterations': 20, 'norm': norm, 'loss': 'epe'},
    {'targeted': False, 'target': np.nan, 'attack': 'pgd', 'epsilon': 0.0314, 'alpha': 0.01, 'iterations': 20, 'norm': norm, 'loss': 'epe'},
    {'targeted': False, 'target': np.nan, 'attack': 'cospgd', 'epsilon': 0.0157, 'alpha': 0.01, 'iterations': 20, 'norm': norm, 'loss': 'epe'},
    {'targeted': False, 'target': np.nan, 'attack': 'cospgd', 'epsilon': 0.0314, 'alpha': 0.01, 'iterations': 20, 'norm': norm, 'loss': 'epe'},
    {'targeted': False, 'target': np.nan, 'attack': 'bim', 'epsilon': 0.0157, 'alpha': 0.01, 'iterations': 20, 'norm': norm, 'loss': 'epe'},
    {'targeted': False, 'target': np.nan, 'attack': 'bim', 'epsilon': 0.0314, 'alpha': 0.01, 'iterations': 20, 'norm': norm, 'loss': 'epe'},
]

# List to store missing combinations
missing_combinations_list = []

# Iterate over the model names that should be present
for model_value in model_names:
    # Check if the model is present in the DataFrame
    model_rows = df_sorted[df_sorted['model'] == model_value]
    
    if model_rows.empty:
        # If the model is missing entirely, add all expected combinations
        for comb in expected_combinations:
            missing_combinations_list.append({
                'model': model_value,
                'dataset': dataset,
                'targeted': comb['targeted'],
                'target': comb['target'],
                'attack': comb['attack'],
                'epsilon': comb['epsilon'],
                'alpha': comb['alpha'],
                'iterations': comb['iterations'],
                'norm': comb['norm'],
                'loss': comb['loss'],
                'model_is_missing': 1  # Model is missing
            })
    else:
        # Otherwise, check for missing combinations
        filtered_rows = model_rows[conditions]
        
        if filtered_rows['targeted'].any():
            for comb in expected_combinations:
                exists = filtered_rows[
                    (filtered_rows['targeted'] == comb['targeted']) & 
                    ((filtered_rows['target'] == comb['target']) | 
                     (filtered_rows['target'].isna() & pd.isna(comb['target']))) & 
                    (filtered_rows['attack'] == comb['attack']) & 
                    (filtered_rows['epsilon'] == comb['epsilon']) &
                    (filtered_rows['alpha'] == comb['alpha']) &
                    (filtered_rows['iterations'] == comb['iterations']) &
                    (filtered_rows['norm'] == comb['norm']) &
                    (filtered_rows['loss'] == comb['loss'])
                ].any().any()
                
                if not exists:
                    missing_combinations_list.append({
                        'model': model_value,
                        'dataset': dataset,
                        'targeted': comb['targeted'],
                        'target': comb['target'],
                        'attack': comb['attack'],
                        'epsilon': comb['epsilon'],
                        'alpha': comb['alpha'],
                        'iterations': comb['iterations'],
                        'norm': comb['norm'],
                        'loss': comb['loss'],
                        'model_is_missing': 0  # Model is present, but combination is missing
                    })

# Create a DataFrame with the missing combinations
missing_combinations_df = pd.DataFrame(missing_combinations_list)

missing_combinations_df_sorted = missing_combinations_df.sort_values(
    by=['model_is_missing', 'model'], 
    ascending=[True, True]  # True means ascending order
)
# Display the resulting DataFrame
print(missing_combinations_df_sorted)
missing_combinations_df_sorted.to_csv(f"missing_iteration_sets_{dataset}_{norm}.csv", index=False)



/scratch/ipykernel_286451/1967594227.py:97: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_rows = model_rows[conditions]
/scratch/ipykernel_286451/1967594227.py:97: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_rows = model_rows[conditions]
/scratch/ipykernel_286451/1967594227.py:97: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_rows = model_rows[conditions]
/scratch/ipykernel_286451/1967594227.py:97: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_rows = model_rows[conditions]
/scratch/ipykernel_286451/1967594227.py:97: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_rows = model_rows[conditions]
/scratch/ipykernel_286451/1967594227.py:97: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_rows = model_rows[conditions]
/scratch/ipykernel_286451/19675942

             model     dataset  targeted    target  attack  epsilon  alpha  \
4           csflow  kitti-2015      True  negative     pgd   0.0314   0.01   
5           csflow  kitti-2015      True      zero     pgd   0.0314   0.01   
6           csflow  kitti-2015      True  negative  cospgd   0.0314   0.01   
7           csflow  kitti-2015      True      zero  cospgd   0.0314   0.01   
8           csflow  kitti-2015      True  negative     bim   0.0314   0.01   
..             ...         ...       ...       ...     ...      ...    ...   
165  videoflow_bof  kitti-2015     False       NaN     pgd   0.0314   0.01   
166  videoflow_bof  kitti-2015     False       NaN  cospgd   0.0157   0.01   
167  videoflow_bof  kitti-2015     False       NaN  cospgd   0.0314   0.01   
168  videoflow_bof  kitti-2015     False       NaN     bim   0.0157   0.01   
169  videoflow_bof  kitti-2015     False       NaN     bim   0.0314   0.01   

     iterations norm loss  model_is_missing  
4            20  

/scratch/ipykernel_286451/1967594227.py:97: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_rows = model_rows[conditions]
/scratch/ipykernel_286451/1967594227.py:97: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_rows = model_rows[conditions]
/scratch/ipykernel_286451/1967594227.py:97: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_rows = model_rows[conditions]
/scratch/ipykernel_286451/1967594227.py:97: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_rows = model_rows[conditions]


In [9]:
import pandas as pd
import numpy as np

# Set the dataset and norm
dataset = "sintel-final"
norm = "inf"

# Define the datasets and corresponding model names
kitti_model_names = [
    "raft", "gma", "rpknet", "ccmr", "craft", "csflow", "dicl", "dip", 
    "fastflownet", "maskflownet", "flow1d", "flowformer", "flowformer++", 
    "gmflow", "gmflownet", "hd3", "irr_pwc", "liteflownet", "liteflownet2", 
    "liteflownet3_pseudoreg", "llaflow", "matchflow", "ms_raft+", 
    "rapidflow", "scopeflow", "scv4", "separableflow", "skflow", 
    "starflow", "videoflow_bof"
]

sintel_model_names = [
    "raft", "pwcnet", "gma", "rpknet", "ccmr", "craft", "dicl", "dip", 
    "fastflownet", "maskflownet", "maskflownet_s", "flow1d", "flowformer", 
    "flowformer++", "gmflow", "hd3", "liteflownet", 
    "liteflownet2", "liteflownet3", "llaflow", "matchflow", 
    "ms_raft+", "rapidflow", "scopeflow", "scv4", "separableflow", 
    "skflow", "starflow", "videoflow_bof"
]

# Choose the model list based on the dataset
if dataset == "kitti-2015":
    model_names = kitti_model_names
elif dataset in ["sintel-clean", "sintel-final"]:
    model_names = sintel_model_names
else:
    raise ValueError(f"Dataset '{dataset}' is not recognized.")

# Read the CSV file
df = pd.read_csv('big_df.csv', delimiter=',')

# Sort by 'model' and filter by 'dataset'
df_sorted = df.sort_values(by='model')
df_sorted = df_sorted[df_sorted["dataset"] == dataset]

# Define expected combinations, including the new criteria
expected_combinations = [
    {'attack': "none"},  # Special case for no attack
    {'targeted': True, 'target': 'negative', 'attack': 'fgsm', 'epsilon': 0.0314, 'loss':"epe", 'norm': norm},
    {'targeted': True, 'target': 'zero', 'attack': 'fgsm', 'epsilon': 0.0314, 'loss':"epe", 'norm': norm},
    {'targeted': False, 'target': np.nan, 'attack': 'fgsm', 'epsilon': 0.0157, 'loss':"epe", 'norm': norm},
    {'targeted': False, 'target': np.nan, 'attack': 'fgsm', 'epsilon': 0.0314, 'loss':"epe", 'norm': norm},
]

# List to store missing combinations
missing_combinations_list = []

# Iterate over the model names that should be present
for model_value in model_names:
    # Check if the model is present in the DataFrame
    model_rows = df_sorted[df_sorted['model'] == model_value]
    
    if model_rows.empty:
        # If the model is missing entirely, add all expected combinations
        for comb in expected_combinations:
            missing_combinations_list.append({
                'model': model_value,
                'dataset': dataset,
                'targeted': comb.get('targeted'),
                'target': comb.get('target'),
                'attack': comb.get('attack'),
                'epsilon': comb.get('epsilon'),
                'loss': comb.get('loss'),
                'norm': comb.get('norm'),
                'model_is_missing': 1  # Model is missing
            })
    else:
        # Otherwise, check for missing combinations
        for comb in expected_combinations:
            if comb['attack'] == "none":
                # Handle the special case where the attack is "none"
                exists = not model_rows[model_rows['attack'] == "none"].empty
            else:
                exists = model_rows[
                    (model_rows['targeted'] == comb.get('targeted')) & 
                    ((model_rows['target'] == comb.get('target')) | 
                     (model_rows['target'].isna() & pd.isna(comb.get('target')))) & 
                    (model_rows['attack'] == comb.get('attack')) & 
                    (model_rows['epsilon'] == comb.get('epsilon')) &
                    (model_rows['loss'] == comb.get('loss')) &
                    (model_rows['norm'] == comb.get('norm'))
                ].any().any()
            
            if not exists:
                missing_combinations_list.append({
                    'model': model_value,
                    'dataset': dataset,
                    'targeted': comb.get('targeted'),
                    'target': comb.get('target'),
                    'attack': comb.get('attack'),
                    'epsilon': comb.get('epsilon'),
                    'loss': comb.get('loss'),
                    'norm': comb.get('norm'),
                    'model_is_missing': 0  # Model is present, but combination is missing
                })

# Create a DataFrame with the missing combinations
missing_combinations_df = pd.DataFrame(missing_combinations_list)
missing_combinations_df_sorted = missing_combinations_df.sort_values(
    by=['model_is_missing', 'model'], 
    ascending=[True, True]  # True means ascending order
)
# Display the resulting DataFrame
print(missing_combinations_df_sorted)
missing_combinations_df_sorted.to_csv(f"missing_sets_{dataset}_{norm}.csv", index=False)

                     model       dataset targeted    target attack  epsilon  \
0                      dip  sintel-final     None      None   none      NaN   
11            flowformer++  sintel-final     None      None   none      NaN   
12            flowformer++  sintel-final     True  negative   fgsm   0.0314   
13            flowformer++  sintel-final     True      zero   fgsm   0.0314   
14            flowformer++  sintel-final    False       NaN   fgsm   0.0157   
15            flowformer++  sintel-final    False       NaN   fgsm   0.0314   
31                ms_raft+  sintel-final     None      None   none      NaN   
32                ms_raft+  sintel-final     True  negative   fgsm   0.0314   
33                ms_raft+  sintel-final     True      zero   fgsm   0.0314   
34                ms_raft+  sintel-final    False       NaN   fgsm   0.0157   
35                ms_raft+  sintel-final    False       NaN   fgsm   0.0314   
46                  skflow  sintel-final     True  n

In [18]:
import pandas as pd
import numpy as np

# Set the dataset and norm
dataset = "sintel-final"
norm = "inf"

# Define the datasets and corresponding model names
kitti_model_names = [
    "raft", "rpknet", "craft", "maskflownet", "flow1d", "flowformer++", 
    "liteflownet3_pseudoreg", "ms_raft+", 
    "scopeflow", "gma"
]

sintel_model_names = [
    "raft", "rpknet", "craft", "maskflownet", "flow1d", "flowformer++", 
    "liteflownet3", "ms_raft+", 
    "scopeflow", "gma"
]

# Choose the model list based on the dataset
if dataset == "kitti-2015":
    model_names = kitti_model_names
elif dataset in ["sintel-clean", "sintel-final"]:
    model_names = sintel_model_names
else:
    raise ValueError(f"Dataset '{dataset}' is not recognized.")

# Read the CSV file
df = pd.read_csv('iteration_df.csv', delimiter=',')

# Replace empty "optim" values with "ground_truth"
df['optim'] = df['optim'].replace(np.nan, 'ground_truth')

# Sort by 'model' and filter by 'dataset' and ensure model is in the model list
df_sorted = df.sort_values(by='model')
df_sorted = df_sorted[(df_sorted["dataset"] == dataset) & (df_sorted["model"].isin(model_names))]

# Define conditions
conditions = (
    (df_sorted['epsilon'].isin([0, 0.0314, 0.0157])) & 
    (df_sorted['alpha'] == 0.01) &
    (df_sorted['iterations'] == 20) &
    (df_sorted['norm'] == norm) &
    (df_sorted['loss'] == 'epe') &
    (df_sorted['targeted'].isin([True, False])) &
    (df_sorted['target'].isin(["zero", "negative"])) | (df_sorted['target'].isna())
)

# Define expected combinations, including the new criteria for "optim"
expected_combinations = [
    {'targeted': True, 'target': 'negative', 'attack': 'pgd', 'epsilon': 0.0314, 'alpha': 0.01, 'iterations': 20, 'norm': norm, 'loss': 'epe', 'optim': "ground_truth"},
    {'targeted': True, 'target': 'zero', 'attack': 'pgd', 'epsilon': 0.0314, 'alpha': 0.01, 'iterations': 20, 'norm': norm, 'loss': 'epe', 'optim': "ground_truth"},
    {'targeted': True, 'target': 'negative', 'attack': 'cospgd', 'epsilon': 0.0314, 'alpha': 0.01, 'iterations': 20, 'norm': norm, 'loss': 'epe', 'optim': "ground_truth"},
    {'targeted': True, 'target': 'zero', 'attack': 'cospgd', 'epsilon': 0.0314, 'alpha': 0.01, 'iterations': 20, 'norm': norm, 'loss': 'epe', 'optim': "ground_truth"},
    {'targeted': True, 'target': 'negative', 'attack': 'bim', 'epsilon': 0.0314, 'alpha': 0.01, 'iterations': 20, 'norm': norm, 'loss': 'epe', 'optim': "ground_truth"},
    {'targeted': True, 'target': 'zero', 'attack': 'bim', 'epsilon': 0.0314, 'alpha': 0.01, 'iterations': 20, 'norm': norm, 'loss': 'epe', 'optim': "ground_truth"},
    {'targeted': False, 'target': np.nan, 'attack': 'pgd', 'epsilon': 0.0157, 'alpha': 0.01, 'iterations': 20, 'norm': norm, 'loss': 'epe', 'optim': "ground_truth"},
    {'targeted': False, 'target': np.nan, 'attack': 'pgd', 'epsilon': 0.0314, 'alpha': 0.01, 'iterations': 20, 'norm': norm, 'loss': 'epe', 'optim': "ground_truth"},
    {'targeted': False, 'target': np.nan, 'attack': 'cospgd', 'epsilon': 0.0157, 'alpha': 0.01, 'iterations': 20, 'norm': norm, 'loss': 'epe', 'optim': "ground_truth"},
    {'targeted': False, 'target': np.nan, 'attack': 'cospgd', 'epsilon': 0.0314, 'alpha': 0.01, 'iterations': 20, 'norm': norm, 'loss': 'epe', 'optim': "ground_truth"},
    {'targeted': False, 'target': np.nan, 'attack': 'bim', 'epsilon': 0.0157, 'alpha': 0.01, 'iterations': 20, 'norm': norm, 'loss': 'epe', 'optim': "ground_truth"},
    {'targeted': False, 'target': np.nan, 'attack': 'bim', 'epsilon': 0.0314, 'alpha': 0.01, 'iterations': 20, 'norm': norm, 'loss': 'epe', 'optim': "ground_truth"},
    # Last two combinations have specific "optim" values
    {'targeted': False, 'target': np.nan, 'attack': 'cospgd', 'epsilon': 0.0314, 'alpha': 0.01, 'iterations': 20, 'norm': norm, 'loss': 'epe', 'optim': "initial_flow"},
    {'targeted': False, 'target': np.nan, 'attack': 'pgd', 'epsilon': 0.0314, 'alpha': 0.01, 'iterations': 20, 'norm': norm, 'loss': 'epe', 'optim': "initial_flow"},
]

# List to store missing combinations
missing_combinations_list = []

# Iterate over the model names that should be present
for model_value in model_names:
    # Check if the model is present in the DataFrame
    model_rows = df_sorted[df_sorted['model'] == model_value]
    
    if model_rows.empty:
        # If the model is missing entirely, add all expected combinations
        for comb in expected_combinations:
            missing_combinations_list.append({
                'model': model_value,
                'dataset': dataset,
                'targeted': comb['targeted'],
                'target': comb['target'],
                'attack': comb['attack'],
                'epsilon': comb['epsilon'],
                'alpha': comb['alpha'],
                'iterations': comb['iterations'],
                'norm': comb['norm'],
                'loss': comb['loss'],
                'optim': comb['optim'],
                'model_is_missing': 1  # Model is missing
            })
    else:
        # Otherwise, check for missing combinations
        filtered_rows = model_rows[conditions]
        
        if filtered_rows['targeted'].any():
            for comb in expected_combinations:
                exists = filtered_rows[
                    (filtered_rows['targeted'] == comb['targeted']) & 
                    ((filtered_rows['target'] == comb['target']) | 
                     (filtered_rows['target'].isna() & pd.isna(comb['target']))) & 
                    (filtered_rows['attack'] == comb['attack']) & 
                    (filtered_rows['epsilon'] == comb['epsilon']) &
                    (filtered_rows['alpha'] == comb['alpha']) &
                    (filtered_rows['iterations'] == comb['iterations']) &
                    (filtered_rows['norm'] == comb['norm']) &
                    (filtered_rows['loss'] == comb['loss']) &
                    (filtered_rows['optim'] == comb['optim'])  # Ensure "optim" matches
                ].any().any()
                
                if not exists:
                    missing_combinations_list.append({
                        'model': model_value,
                        'dataset': dataset,
                        'targeted': comb['targeted'],
                        'target': comb['target'],
                        'attack': comb['attack'],
                        'epsilon': comb['epsilon'],
                        'alpha': comb['alpha'],
                        'iterations': comb['iterations'],
                        'norm': comb['norm'],
                        'loss': comb['loss'],
                        'optim': comb['optim'],
                        'model_is_missing': 0  # Model is present, but combination is missing
                    })

# Create a DataFrame with the missing combinations
missing_combinations_df = pd.DataFrame(missing_combinations_list)

# Sort the DataFrame by 'model_is_missing' and 'model'
missing_combinations_df_sorted = missing_combinations_df.sort_values(
    by=['model_is_missing', 'model'], 
    ascending=[True, True]  # True means ascending order
)

# Display the resulting DataFrame
print(missing_combinations_df_sorted)
missing_combinations_df_sorted.to_csv(f"missing_top_ten_sets_{dataset}_{norm}.csv", index=False)


/scratch/ipykernel_629528/3185763964.py:96: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_rows = model_rows[conditions]
/scratch/ipykernel_629528/3185763964.py:96: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_rows = model_rows[conditions]
/scratch/ipykernel_629528/3185763964.py:96: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_rows = model_rows[conditions]
/scratch/ipykernel_629528/3185763964.py:96: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_rows = model_rows[conditions]
/scratch/ipykernel_629528/3185763964.py:96: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_rows = model_rows[conditions]
/scratch/ipykernel_629528/3185763964.py:96: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_rows = model_rows[conditions]
/scratch/ipykernel_629528/31857639

   model       dataset  targeted  target  attack  epsilon  alpha  iterations  \
0  craft  sintel-final     False     NaN  cospgd   0.0314   0.01          20   
1  craft  sintel-final     False     NaN     pgd   0.0314   0.01          20   

  norm loss         optim  model_is_missing  
0  inf  epe  initial_flow                 0  
1  inf  epe  initial_flow                 0  
